In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
from sklearn import preprocessing

In [ ]:
# NBA season we will be analyzing
year = 2019
# URL page we will scraping (see image above)
url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html".format(year)
# this is the HTML from the given URL
html = urlopen(url)
soup = BeautifulSoup(html)

# use findALL() to get the column headers
soup.findAll('tr', limit=2)
# use getText()to extract the text we need into a list
headers = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]
# exclude the first column as we will not need the ranking order from Basketball Reference for the analysis
headers = headers[1:]

# avoid the first header row
rows = soup.findAll('tr')[1:]
player_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]
stats = pd.DataFrame(player_stats, columns = headers)
stats.drop_duplicates(subset='Player', keep='first',inplace=True)
stats.set_index("Player",inplace=True)
stats.drop('Tm', 1, inplace=True)

In [ ]:
stats = pd.get_dummies(stats, columns=['Pos'])

In [ ]:
stats = stats.apply(pd.to_numeric)
corr = stats.corr()
#corr.to_csv('output.csv', encoding='utf-8', index=False)

threshold = 0.75

columns = np.full((corr.shape[0],), True, dtype=bool)
for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i,j] >= threshold:
            if columns[j]:
                columns[j] = False
selected_columns = stats.columns[columns]
stats = stats[selected_columns]
selected_columns


In [ ]:
mvp = []
for i in range(531):
    if i == 18:
        mvp.append(1)
    else :
        mvp.append(0)
scaler = preprocessing.StandardScaler()
scaler.fit(stats)
x_scaled = scaler.transform(stats)
stats
stats['MVP'] = mvp
# stats.dtypes